In [ ]:
from gprm.datasets import Reconstructions, Zircons, Rocks
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pygmt

import matplotlib as mpl
mpl.rc('font',family='Helvetica')
mpl.rcParams['axes.linewidth'] = 2
mpl.rcParams['xtick.major.width'] = 2
mpl.rcParams['ytick.major.width'] = 2

import sys
#sys.path.append('/Users/simon/OneDrive/Andes_works//python/')
sys.path.append('../../andes_paper/python/')
import joyful_geochemistry as joy
import joyful_mapping as joymap

%load_ext autoreload
%autoreload 2


In [ ]:
M2022 = Reconstructions.fetch_Muller2022()

gdf_Hf_M2022 = M2022.assign_plate_ids(gdf_Hf, polygons='continents', keep_unpartitioned_features=False)

In [ ]:
time_step = 20

for reconstruction_time in np.arange(0,1000,time_step):

    tmp = gdf_Hf_M2022.query('Age>@reconstruction_time-@time_step/2 & Age<@reconstruction_time+@time_step/2')
    
    tmp = M2022.reconstruct(tmp, reconstruction_time)
    
    result = pygmt.blockmedian(x=tmp.geometry.x, y=tmp.geometry.y, z=tmp.Hf, region='d', spacing='2.5d', C=True)
    

    title = '{:0.0f} Ma'.format(reconstruction_time)
    
    fig = pygmt.Figure()
    fig.basemap(projection='N25c', region='d', frame=["af", f"wsne+t{title}"])
    #fig.coast(land='grey')
    M2022.polygon_snapshot('continents', reconstruction_time+0.001).plot(fig, fill='grey80', pen='0p,gray80')
    #M2022.plate_snapshot(reconstruction_time+0.01).plot_boundaries(fig, pen='0.5p,darkgrey')
    plates = M2022.plate_snapshot(reconstruction_time)
    plates.plot_subduction_zones(fig, pen='1p,darkgrey', fill='darkgrey')
    plates.plot_mid_ocean_ridges(fig, pen='1p,darkgrey')
    pygmt.makecpt(cmap='polar', series=[-15,15,1], background='o')
    fig.plot(x=result.iloc[:,0], y=result.iloc[:,1], fill=result.iloc[:,2], style='c0.18c', pen='0.05p,black', cmap=True)
    fig.savefig('../images/sequence_M2021/global_hafnium_{:0.0f}Ma.png'.format(reconstruction_time))
    #fig.show(width=900)
    
    #break